In [105]:
import pandas as pd 
import numpy as np
from numpy import array
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
from ipynb.fs.full.evaluation import *
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from ipynb.fs.full.Data_Processing import *
from sklearn import preprocessing
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
torch.set_printoptions(linewidth=120) #Display options for output
torch.set_grad_enabled(True) # Already on by default
torch.manual_seed(0)
from torch_lr_finder import LRFinder
import pickle
import torch.utils.data as data_utils
from collections import namedtuple
import time
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [106]:
import numpy as np
import torch
# https://github.com/Bjarten/early-stopping-pytorch
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [107]:
class EEGNet(nn.Module):
    def __init__(self, num_classes, combined, fc_size,dropout):
        super(EEGNet, self).__init__()
        self.T = 120
        self.combined = combined
        self.fc_size = fc_size
        self.dropout = dropout
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 8), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints.
    
        self.fc1 = nn.Linear(fc_size, num_classes)
        
    def forward(self, x):
        # Layer 1
        x = x.float()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, self.dropout)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, self.dropout)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, self.dropout)
        x = self.pooling3(x)
 
        # FC Layer
        x = x.view(-1, self.fc_size)
        if self.combined == False:      
            x = self.fc1(x)
        return x
    

    
    # for 60 timepoints = 4*2*4 and -1
    # 120 timepoints = 4* 2* 7 and -1
    # https://discuss.pytorch.org/t/runtimeerror-shape-1-400-is-invalid-for-input-of-size/33354
    # https://discuss.pytorch.org/t/valueerror-expected-input-batch-size-324-to-match-target-batch-size-4/24498/2

In [108]:
class Combine(nn.Module):
    def __init__(self, num_classes, combined, fc_size,dropout):
        super(Combine, self).__init__()
        self.cnn = EEGNet(num_classes,combined, fc_size,dropout)
        self.rnn = nn.LSTM(
            input_size=fc_size, 
            hidden_size=16, 
            num_layers=1,
            batch_first=True)
        self.linear = nn.Linear(16 ,num_classes)



    def forward(self, x):
        batch_size, C, timepoints, channels = x.size()
        c_in = x
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, -1 , c_out.shape[1])
        r_out, (h_n, h_c) = self.rnn(r_in)
        r_out2 = self.linear(r_out[:, -1, :])
        return F.log_softmax(r_out2, dim=1)
     

In [109]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()### Scale the data

In [110]:
def learning_rate_finder(model,train_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-7, weight_decay=1e-2)
    lr_finder = LRFinder(model, optimizer, criterion)
    lr_finder.range_test(train_loader, end_lr=100, num_iter=100, step_mode="exp")
    lr_finder.plot()
    plt.savefig("HackX/results/figures/learning_rate_finder")

In [111]:
def get_accuracy(loader, net):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs = data[0].to(device)
            labels = data[1].to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = (correct/total * 100)
    return acc

In [112]:
def predict(loader, net):
    correct = 0
    total = 0
    predictions = []
    with torch.no_grad():
        for data in loader:
            inputs = data[0].to(device)
            labels = data[1].to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            predictions.append(predicted.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = (correct/total * 100)
    return np.concatenate(predictions)

In [113]:
def train(X_train, train_loader, valid_loader,  num_classes, model, n_epochs, patience, train_verbose, fc_size,dropout,model_type):
    
    # choose between the EEGNet or EEGNet + RNN
    if model == 'EEGNet':
        net = EEGNet(num_classes, False, fc_size,dropout).to(device)
    if model == 'Hybrid':
        net = Combine(num_classes, True, fc_size,dropout).to(device)
        
    #store the predictions and the losses
    preds_list = [] # track the predictions
    labels_list = [] # track the labels
    train_losses = [] # to track the train loss as the model trains
    valid_losses = [] # to track the validation loss as the model trains
    avg_train_losses = [] # to track the average training loss per epoch as the model trains
    avg_valid_losses = [] # to track the average validation loss per epoch as the model trains
    
    #Set the optimiser 
    optimizer = optim.Adam(net.parameters(), lr = 0.001)
    #Initialise the early stopping
    early_stopping = EarlyStopping(patience=patience, verbose=train_verbose, path='checkpoint3.pt')

    #Implement learning rate finder
#     learning_rate_finder(net,train_loader)
    
    for epoch in range(n_epochs):
        total_loss = 0
        total_correct = 0

        net.train() # prep the model for training
        for batch in train_loader:

            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            
            preds = net(inputs) #forward pass: compute predicted outputs by passing inputs to the model
        
    
#             criterion=nn.BCEWithLogitsLoss() # calculate the loss
            if model_type == 'clf':
                loss = F.cross_entropy(preds, labels.long()) # calculate loss
            else:
                loss_func = nn.MSELoss()
                loss = loss_func(preds,labels.float())
            optimizer.zero_grad()# clear the gradients of all optimized variables
            loss.backward()  # backward pass: compute gradient of the loss with respect to model parameters
            optimizer.step() # perform a single optimization step (parameter update)
            train_losses.append(loss.item()) # record training loss
            
            #record the predictions and losses
            preds_list.append(preds)
            labels_list.append(labels)
            total_loss += loss.item()
            if model == 'clf':
                total_correct += get_num_correct(preds, labels)
            
        ######################    
        # validate the model #
        ######################
        net.eval() # prep model for evaluation
        for batch in valid_loader:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            
            # forward pass: compute predicted outputs by passing inputs to the model
            preds = net(inputs)
            # calculate the loss
            if model_type == 'clf':
                loss = F.cross_entropy(preds, labels.long()) # calculate loss
            else:
                loss_func = nn.MSELoss()
                loss = loss_func(preds,labels.float())
            # record validation loss
            valid_losses.append(loss.item())

        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'epoch: [{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f} '+
                     f'train_accuracy: {total_correct/len(X_train):.5f}' )
        
        if train_verbose == True: print(print_msg)
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, net)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break       
        
    # load the last checkpoint with the best model
    net.load_state_dict(torch.load('checkpoint3.pt'))
    
    return net, avg_train_losses, avg_valid_losses

In [114]:
def save_plots(model_type, y_true, y_pred, user, label, n_epochs, model, eval_type, train_loss, valid_loss, bandpass, multiple,sigma, class_type):
    if model_type == 'clf':
        # plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        saved_file = "results/CNN/clf/confusion/k fold/{3}/{4}/User_{0}_{1}_epochs_{2}_bandpass_{5}_multiple_{6}_sigma_{7}_{8}.png".format(user,label,
                                                                                                                            n_epochs,model, 
                                                                                                                            eval_type, 
                                                                                                                            bandpass, multiple,sigma, class_type)
        plot_confusion_matrix(cm, set(y_true), saved_file ,normalize=True)
        
        #plot model
    if model_type == 'reg':
        saved_file = "results/CNN/reg/y vs y_pred/{3}/{4}/User_{0}_{1}_epochs_{2}_bandpass_{5}_multiple_{6}_sigma_{7}_{8}.png".format(user,label, n_epochs,model,
                                                                                                                                  eval_type, 
                                                                                                                                  bandpass, multiple,sigma, class_type)
        plot_model(y_true, y_pred, user, label,file=saved_file)
    
    saved_file = "results/CNN/clf/loss curves/k fold/{3}/{4}/User_{0}_{1}_epochs_{2}_bandpass_{5}_multiple_{6}_sigma_{7}{8}.png".format(user,label,n_epochs,
                                                                                                                                     model, eval_type, bandpass,
                                                                                                                                    multiple,sigma, class_type)
    plot_loss_early_stop(train_loss, valid_loss, saved_file)

In [115]:
def get_loaders(X_train, X_valid, y_train, y_valid):
    
    #Convert to 4D 
    X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
    X_valid = X_valid.reshape(X_valid.shape[0],1, X_valid.shape[1],X_valid.shape[2])

    # Create train and valid loader
    train = data_utils.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
    valid = data_utils.TensorDataset(torch.Tensor(X_valid), torch.Tensor(y_valid))
    train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)
    valid_loader = data_utils.DataLoader(valid, batch_size=50, shuffle=False)       
    return train_loader, valid_loader

In [116]:
def multiply(data, multiple):
    """
    Method for multiplying a dataset
    :data: chosen dataset
    :multiply: chosen number to multiply the dataset
    """
    data_list = []
    for i in range (multiple):
        data_list.append(data)
    data = np.concatenate(data_list)
    return data


# Gaussian noise

# https://het.as.utexas.edu/HET/Software/Numpy/reference/generated/numpy.random.normal.html

def add_gaussian_noise(clean_signal, multiple, sigma):
    """
    Method for multiplying a given dataset and adding gausian noise
    :clean_signal: clean dataset without noise
    :multiple: chosen number to multiply the dataset
    :sigma: standard deviation
    """
    print("Starting size: {0}".format(clean_signal.shape))
    #multiply the dataset
    clean_signal = multiply(clean_signal, multiple)

    # add noise to the dataset based on the shape of the multiplied dataset
    mu = 0 # average needs to be zero to generate gaussian noise
    noise = np.random.normal(mu, sigma, clean_signal.shape)
    noisy_signal = clean_signal + noise
    print("End size: {0}".format(noisy_signal.shape))
    return noisy_signal

In [117]:
def kfold_predict(X,y, model_type, model, n_epochs, train_verbose, patience, fc_size, multiple, sigma, augment, class_type,dropout):
    """
    Method for running 5 fold cross validation based on a given array of tests
    """
    kf= KFold(n_splits = 5, shuffle = True, random_state =  1)
    
    if model_type == 'clf':
        results = {"Accuracy":[], "Precision":[], "Recall":[], "F1 Score Macro":[],
              "F1 Score Micro":[],"Balanced Accuracy":[]}
    else:
        results = {'RMSE':[], 'R2':[]}
        
    total_predictions = []
    total_true = []
    num_classes = 0
    accuracy = []
    fold = 0
    for train_index, test_index in kf.split(X):
        
        #Train/valid split
        X_train, X_valid = np.concatenate(X[train_index]), np.concatenate(X[test_index])
        y_train, y_valid = np.concatenate(y[train_index]).astype('int'), np.concatenate(y[test_index]).astype('int')
        
        
        # check the the classes in the validation set, if there are not in training set then skip
        y_valid_classes = list(set(y_valid))
        y_train_classes = list(set(y_train))
        
        if check_if_valid_labels_are_in_train(y_train_classes, y_valid_classes) == False: 
            continue
   
        #standardise per channel
        X_train, X_valid = standardise(X_train, X_valid)
        
        #convert to binary if binary classification
        if class_type == 'binary':
            y_train = convert_to_binary(y_train)
            y_valid = convert_to_binary(y_valid)
                
        if model_type == 'clf':
        #label the categorical variables
            y_train, y_valid, le = categorise(y_train, y_valid)
        
        # augment training samples
        if augment == True:
            X_train = add_gaussian_noise(X_train, multiple, sigma)
            y_train = multiply(y_train, multiple)
        
        size = len(X_train) + len(X_valid)
        #get loaders
        train_loader, valid_loader = get_loaders(X_train, X_valid, y_train, y_valid)
        
         # count the number of classes
        if len(set(y_train)) > num_classes:
            num_classes = len(set(y_train))
       
        # train the network
        time_start = time.time()
        net, train_loss, valid_loss = train(X_train, train_loader, valid_loader, 
                    num_classes, model, n_epochs, patience, train_verbose, fc_size,dropout, model_type)
        fold += 1 
        print('Fold  {0}! Time elapsed: {1} seconds'.format(fold, time.time()-time_start))
        
        # make predictions
        if model_type == 'clf':        
            y_pred = le.inverse_transform(predict(valid_loader, net))
            y_true = le.inverse_transform(y_valid)
        else:
            y_pred = predict(valid_loader, net)
            y_true = y_valid
     
        #save total predictions and get results
        total_predictions.append(y_pred)
        total_true.append(y_true) 
        r = get_results(y_true, y_pred, model_type) #returns a dictionary of results   
        valid_acc = get_accuracy(valid_loader, net)
        for key in r: # loop through dictionary to add to all the scores to the results dictionary
            results[key].append(r[key])
        accuracy.append(valid_acc)

    for key in results: # average out the results 
        results[key] = average(results[key])
    accuracy = average(accuracy)
    total_predictions = np.concatenate(total_predictions)
    total_true = np.concatenate(total_true)
        
        
    return results, total_predictions , total_true , num_classes , size , accuracy, train_loss, valid_loss

In [120]:
def run_per_user(model , train_verbose, window_size_samples, fc_size, bandpass, multiple, sigma, augment,class_type, dropout):

    time_original = time.time()

    labels = ["attention", "interest", "effort"]

    n_epochs = 100

    patience = 10
    window_size_samples = 120
    saved_file = "all_users_sampled_{0}_window_annotated_EEG_no_agg_bandpass_{1}_slider_{0}.pickle".format(window_size_samples, bandpass)
    all_tests = load_file(saved_file)
    users = list(all_tests.keys())
    model_type = 'clf'
    results = []
    eval_type = 'per user'
    
    for user in users:
        print("Working on user {0}".format(user))

        for label in labels:

            time_start = time.time()
            dt = all_tests[user] # dictionary of all the individual tests per user

            X = np.array([np.array(x).astype(np.float32) for x in dt['inputs']]) # array of all the inputs for each test
            y = np.array([np.array(x) for x in dt[label]]) #Convert the categories into labels

            # K fold predict 
            r, y_pred, y_true, num_classes, size, accuracy, train_loss, valid_loss = kfold_predict(X,y, model_type, 
                                                                                                   model, n_epochs, 
                                                                                                    train_verbose, patience, 
                                                                                                   fc_size, multiple, sigma, augment, class_type, dropout)
   
            # get results and add them to the list
            duration = time.time() - time_start
            results.append(collate_results(r, user, label, duration, num_classes, 
                                           size, model_type, n_epochs, window_size_samples, model, multiple, sigma, bandpass,class_type))

            #Save plots
            save_plots(model_type, y_true, y_pred, user, label, n_epochs, model, eval_type, train_loss, valid_loss, bandpass, multiple,sigma , class_type)

            print("Finished analysis on User {0}_{1}".format(user,label))
        print("Finished analysis on User {0}".format(user))
    results  = pd.DataFrame(results)
    results.to_csv("results/CNN/{3}/tabulated/k fold/{2}/{2}_performance_window_size_{0}_{1}_{4}_multiple_{5}_sigma{6}_{7}.csv".format(window_size_samples, 
                                                                                                                                       n_epochs, model, model_type, 
                                                                                                                                       eval_type, multiple, sigma, class_type), index=False )
    final_duration = time.time()- time_original
    print("All analyses are complete! Time elapsed: {0}".format(final_duration))
    return results
        
    
def run_cross_user(model , train_verbose, window_size_samples, fc_size, bandpass, multiple, sigma, augment,class_type, dropout, model_type):
    time_original = time.time()

#     labels = ["attention", 'interest', 'effort']
    labels = ['attention','interest','effort']
    results = [] # save all results in this list
    patience = 10
    n_epochs = 500
    saved_file = "saved user and test data/all_users_sampled_{0}_window_annotated_EEG_agg_bandpass_{1}_slider_{0}.pickle".format(window_size_samples, bandpass)
    all_tests_agg = load_file(saved_file)
    users = all_tests_agg.keys()

    user ='all'
    eval_type = 'cross user'
   

    for label in labels:
         print("Running - Model_type: {4}, ClassType:{3}, Model: {0}, User: {1}, label: {2}".format(model_type, user,label, class_type, model_type))
         time_start = time.time()

         # Store each user in a list to prepare for cross-user analysis
         X = np.array([all_tests_agg[user]['inputs'].astype(np.float32) for user in all_tests_agg])
         y = np.array([all_tests_agg[user][label] for user in all_tests_agg])  

         # train and make predictions
         r, y_pred, y_true, num_classes, size, accuracy, train_loss, valid_loss = kfold_predict(X,y,model_type, model,
                                                                                               n_epochs, train_verbose, 
                                                                                               patience, fc_size, multiple,sigma, augment, class_type, dropout)

         # get results
         duration = time.time() - time_start
         results.append(collate_results(r, user, label, duration, num_classes, 
                                       size, model_type, n_epochs, window_size_samples,model, multiple, sigma, bandpass, class_type))
        
         #Save plots
         save_plots(model_type, y_true, y_pred, user, label, n_epochs, model, eval_type, train_loss, valid_loss, bandpass, multiple,sigma, class_type)
                 
         print("Finished analysis on label {0}. Time elapsed {1}".format(label, time.time()-time_start))
    print("Finished analysis on User {0}".format(user))
    results_file = "results/CNN/{3}/tabulated/k fold/{2}/{2}_performance_window_size_{0}_{1}_{4}_dropout_{9}_all_labels_bandpass_{5}_multiple_{6}_sigma_{7}_{8}.csv".format(window_size_samples, n_epochs, model,
                                                                                                                                model_type, eval_type, bandpass, multiple, sigma, class_type,dropout)
    results  = pd.DataFrame(results)
    results.to_csv(results_file, index=False )
    final_duration = time.time()- time_original
    print("All analyses are complete! Time elapsed: {0}".format(final_duration))
    return results

# windows = [15, 30, 60, 120, 250]
# fc_sizes = [8, 16, 32, 56, 120]
# for window, fc_size in zip(windows, fc_sizes):

window = 120
fc_size = 56
augment = False
multiples = [5, 20, 30]
sigmas = [0.001, 0.01, 0.1, 0.5]
# dropouts = [0.1, 0.25, 0.50, 0.75, 0.8, 0.9]
dropout = 0.9
bandpass = False
class_type = 'binary'
multiple = None 
sigma = None
model = 'Hybrid' # or "EEGNet"
model_type = 'clf'
# models= 'Hybrid'
results = []


run_cross_user(model, False, window, fc_size,bandpass, multiple, sigma, augment, class_type, dropout, model_type)



        

FileNotFoundError: [Errno 2] No such file or directory: 'saved user and test data/all_users_sampled_120_window_annotated_EEG_agg_bandpass_False_slider_120.pickle'